<a href="https://colab.research.google.com/github/lukekolbe/AL-in-CreditScoring/blob/main/result_processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep Runtime

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
!pip freeze

absl-py==1.0.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.12.0
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.4
Babel==2.10.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.4
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2021.10.8
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.22.4
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.28
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.4
distributed=

In [3]:
!pip uninstall matplotlib -y

Found existing installation: matplotlib 3.2.2
Uninstalling matplotlib-3.2.2:
  Successfully uninstalled matplotlib-3.2.2


In [4]:
!pip install pingouin
!pip install scikit_posthocs
!pip install matplotlib

     |████████████████████████████████| 183 kB 7.2 MB/s 
     |████████████████████████████████| 38.1 MB 1.6 MB/s 
     |████████████████████████████████| 11.2 MB 17.0 MB/s 
     |████████████████████████████████| 9.8 MB 41.5 MB/s 
     |████████████████████████████████| 930 kB 41.4 MB/s 
  Created wheel for pingouin: filename=pingouin-0.5.1-py3-none-any.whl size=194002 sha256=92e75a916e86a2795f90c46618ed52a14ae9df3c09f13d5ef57c993c2faae3fc
  Stored in directory: /root/.cache/pip/wheels/74/a7/b6/5b5f03b0395fdada1f45d3aed76e7da48873a02805545ef652
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=0e8842178b6fbf9a78e46322c4ba543843994348dd5a36c7883626d1caf04bf2
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built pingouin littleutils
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-

In [5]:
############ LIBRARIES

import os
import time
import datetime
import random
import pickle
import re
import copy
import gc
import sys
import json

gc.enable()

import warnings
warnings.filterwarnings('ignore')

import pingouin as pg
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
plt.style.use('default')
%matplotlib inline
import seaborn as sns

import scipy.stats
from collections import namedtuple
import scikit_posthocs as sp

In [6]:
os.chdir('/gdrive/My Drive/ACTIVE LEARNING THESIS/')

# Helper Functions

In [7]:
from collections import namedtuple

'''
  This function is an enhanced version of the Friedman Test found in the popular scipy package.
  The added functionality computes z-values (pivotal quantities) that are needed to run the Holm-Test, i.e.
  pairwise comparisons of each model to the control model, with p-value adjustment.
  The formula for the z-statistic can be found in DEMSAR (2006).
'''

#FriedmanchisquareResult = namedtuple('FriedmanchisquareResult',
#                                     ('statistic', 'pvalue'))

def friedmanchisquare2(*args):
    """Compute the Friedman test for repeated measurements.
    The Friedman test tests the null hypothesis that repeated measurements of
    the same individuals have the same distribution.  It is often used
    to test for consistency among measurements obtained in different ways.
    For example, if two measurement techniques are used on the same set of
    individuals, the Friedman test can be used to determine if the two
    measurement techniques are consistent.
    Parameters
    ----------
    measurements1, measurements2, measurements3... : array_like
        Arrays of measurements.  All of the arrays must have the same number
        of elements.  At least 3 sets of measurements must be given.
    Returns
    -------
    statistic : float
        The test statistic, correcting for ties.
    pvalue : float
        The associated p-value assuming that the test statistic has a chi
        squared distribution.
    
    Notes
    -----
    Due to the assumption that the test statistic has a chi squared
    distribution, the p-value is only reliable for n > 10 and more than
    6 repeated measurements.
    References
    ----------
    .. [1] https://en.wikipedia.org/wiki/Friedman_test
    """

    k = len(args)
    if k < 3:
        raise ValueError('At least 3 sets of measurements must be given '
                         'for Friedman test, got {}.'.format(k))

    n = len(args[0])
    for i in range(1, k):
        if len(args[i]) != n:
            raise ValueError('Unequal N in friedmanchisquare.  Aborting.')

    # Rank data
    data = np.vstack(args).T
    data = data.astype(float)
    for i in range(len(data)):
        data[i] = scipy.stats.rankdata(data[i])

    # Handle ties
    ties = 0
    for d in data:
        replist, repnum = scipy.stats.find_repeats(np.array(d))
        for t in repnum:
            ties += t * (t*t - 1)
    c = 1 - ties / (k*(k*k - 1)*n)
    

    ssbn = np.sum(data.sum(axis=0)**2)
    chisq = (12.0 / (k*n*(k+1)) * ssbn - 3*n*(k+1)) / c #test statistic
    pvalue = scipy.stats.distributions.chi2.sf(chisq, k - 1) #p-value

    ########## ADDED CUSTOM FUNCTIONALITY TO RUN HOLM TEST ##########
    #print('data from function', data)
    rankings_avg = [np.mean([case[j] for case in data]) for j in range(k)]
    rankings_cmp = [r/np.sqrt(k*(k+1)/(6.*n)) for r in rankings_avg] #test statistics (z-value) for each method (DEMSAR 2006)
    ##########

    return chisq, pvalue, data, rankings_avg, rankings_cmp

In [8]:
# ALTERNATIVE VERSION OF FRIEDMAN TEST
# APPLIES IMAN-DAVENPORT CORRECTION TO P-VALUES

def friedman_test(*args):

    """
        Performs a Friedman ranking test.
        Tests the hypothesis that in a set of k dependent samples groups (where k >= 2) at least two of the groups represent populations with different median values.
        
        Parameters
        ----------
        sample1, sample2, ... : array_like
            The sample measurements for each group.
            
        Returns
        -------
        F-value : float
            The computed F-value of the test.
        p-value : float
            The associated p-value from the F-distribution.
        rankings : array_like
            The ranking for each group.
        pivots : array_like
            The pivotal quantities for each group.
            
        References
        ----------
        M. Friedman, The use of ranks to avoid the assumption of normality implicit in the analysis of variance, Journal of the American Statistical Association 32 (1937) 674–701.
        D.J. Sheskin, Handbook of parametric and nonparametric statistical procedures. crc Press, 2003, Test 25: The Friedman Two-Way Analysis of Variance by Ranks
    """

    k = len(args)
    if k < 2: raise ValueError('Less than 2 levels')
    n = len(args[0])
    if len(set([len(v) for v in args])) != 1: raise ValueError('Unequal number of samples')

    rankings = []
    for i in range(n):
        row = [col[i] for col in args]
        row_sort = sorted(row)
        rankings.append([row_sort.index(v) + 1 + (row_sort.count(v)-1)/2. for v in row])

    rankings_avg = [np.mean([case[j] for case in rankings]) for j in range(k)]
    rankings_cmp = [r/np.sqrt(k*(k+1)/(6.*n)) for r in rankings_avg] #see demsar (2006)

    chi2 = ((12*n)/float((k*(k+1))))*((sum(r**2 for r in rankings_avg))-((k*(k+1)**2)/float(4))) #(uncorrected) chi-square test statistic
    iman_davenport = ((n-1)*chi2)/float((n*(k-1)-chi2)) #corrected test-statistic

    p_value = 1 - scipy.stats.f.cdf(iman_davenport, k-1, (k-1)*(n-1)) #iman-davenport corrected p-value
    
    ###########
    p_value_uncorrected = scipy.stats.distributions.chi2.sf(chi2, k - 1)
    ###########

    return iman_davenport,p_value, chi2, p_value_uncorrected, rankings, rankings_avg, rankings_cmp

In [9]:
import itertools as it

def holm_test(pivots, control=None):
    """
        Performs a Holm post-hoc test using the pivot quantities obtained by a ranking test.
        Tests the hypothesis that the ranking of the control method is different to each of the other methods.
        
        Parameters
        ----------
        pivots : dictionary_like
            A dictionary with format 'groupname':'pivotal quantity' 
        control : string optional
            The name of the control method (one vs all), default None (all vs all) 
            
        Returns
        ----------
        comparions : array-like
            Strings identifier of each comparison with format 'group_i vs group_j'
        Z-values : array-like
            The computed Z-value statistic for each comparison.
        p-values : array-like
            The associated p-value from the Z-distribution wich depends on the index of the comparison
        adj_p-values : array-like
            The associated adjusted p-values wich can be compared with a significance level
            
        References
        ----------
        O.J. S. Holm, A simple sequentially rejective multiple test procedure, Scandinavian Journal of Statistics 6 (1979) 65–70.
    """
    k = len(pivots.keys())
    values = list(pivots.values())
    keys = list(pivots.keys())
    if not control :
        control_i = values.index(min(values))
    else:
        control_i = keys.index(control)

    comparisons = [keys[control_i] + " vs " + keys[i] for i in range(k) if i != control_i]
    z_values = [abs(values[control_i] - values[i]) for i in range(k) if i != control_i]
    p_values = [2*(1-scipy.stats.norm.cdf(abs(z))) for z in z_values]
    
    # Sort values by p_value so that p_0 < p_1
    p_values, z_values, comparisons = map(list, zip(*sorted(zip(p_values, z_values, comparisons), key=lambda t: t[0])))
    adj_p_values = [min(max((k-(j+1))*p_values[j] for j in range(i+1)), 1) for i in range(k-1)]
    
    return comparisons, z_values, p_values, adj_p_values

In [10]:
def finner_test(pivots, control=None):
    """
        Performs a Finner post-hoc test using the pivot quantities obtained by a ranking test.
        Tests the hypothesis that the ranking of the control method is different to each of the other methods.
        
        Parameters
        ----------
        pivots : dictionary_like
            A dictionary with format 'groupname':'pivotal quantity' 
        control : string optional
            The name of the control method,  default the group with minimum ranking
            
        Returns
        ----------
        comparions : array-like
            Strings identifier of each comparison with format 'group_i vs group_j'
        Z-values : array-like
            The computed Z-value statistic for each comparison.
        p-values : array-like
            The associated p-value from the Z-distribution wich depends on the index of the comparison
        adj_p-values : array-like
            The associated adjusted p-values wich can be compared with a significance level
            
        References
        ----------
        H. Finner, On a monotonicity problem in step-down multiple test procedures, Journal of the American Statistical Association 88 (1993) 920–923.
    """
    
    k = len(pivots)
    values = list(pivots.values())
    keys = list(pivots.keys())
    if not control :
        control_i = values.index(min(values))
    else:
        control_i = keys.index(control)

    comparisons = [keys[control_i] + " vs " + keys[i] for i in range(k) if i != control_i]
    z_values = [abs(values[control_i] - values[i]) for i in range(k) if i != control_i]
    p_values = [2*(1-scipy.stats.norm.cdf(abs(z))) for z in z_values]
    # Sort values by p_value so that p_0 < p_1
    p_values, z_values, comparisons = map(list, zip(*sorted(zip(p_values, z_values, comparisons), key=lambda t: t[0])))
    adj_p_values = [min(max(1-(1-p_values[j])**((k-1)/float(j+1)) for j in range(i+1)), 1) for i in range(k-1)]
    
    return comparisons, z_values, p_values, adj_p_values

In [11]:
def bonferroni_dunn_test(pivots, control=None):
    """
        Performs a Bonferroni-Dunn post-hoc test using the pivot quantities obtained by a ranking test.
        Tests the hypothesis that the ranking of the control method is different to each of the other methods.
        
        Parameters
        ----------
        pivots : dictionary_like
            A dictionary with format 'groupname':'pivotal quantity' 
        control : string optional
            The name of the control method (one vs all), default None (all vs all) 
            
        Returns
        ----------
        comparions : array-like
            Strings identifier of each comparison with format 'group_i vs group_j'
        Z-values : array-like
            The computed Z-value statistic for each comparison.
        p-values : array-like
            The associated p-value from the Z-distribution wich depends on the index of the comparison
        adj_p-values : array-like
            The associated adjusted p-values wich can be compared with a significance level
            
        References
        ----------
        O.J. Dunn, Multiple comparisons among means, Journal of the American Statistical Association 56 (1961) 52–64.
    """
    k = len(pivots)
    values = list(pivots.values())
    keys = list(pivots.keys())
    if not control :
        control_i = values.index(min(values))
    else:
        control_i = keys.index(control)

    comparisons = [keys[control_i] + " vs " + keys[i] for i in range(k) if i != control_i]
    z_values = [abs(values[control_i] - values[i]) for i in range(k) if i != control_i]
    p_values = [2*(1-scipy.stats.norm.cdf(abs(z))) for z in z_values]
    # Sort values by p_value so that p_0 < p_1
    p_values, z_values, comparisons = map(list, zip(*sorted(zip(p_values, z_values, comparisons), key=lambda t: t[0])))
    adj_p_values = [min((k-1)*p_value,1) for p_value in p_values]
    
    return comparisons, z_values, p_values, adj_p_values

In [12]:
# Helper functions for performing the statistical tests
def generate_scores(method, method_args, data, labels):
    pairwise_scores = method(data, **method_args) # Matrix for all pairwise comaprisons
    pairwise_scores.set_axis(labels, axis='columns', inplace=True) # Label the cols
    pairwise_scores.set_axis(labels, axis='rows', inplace=True) # Label the rows, note: same label as pairwise combinations
    return pairwise_scores

def pairwise_plotter(scores):
    # Pretty plot of significance
    heatmap_args = {'linewidths': 0.25, 'linecolor': '0.5', 'square': True,
                    'cbar_ax_bbox': [0.80, 0.35, 0.04, 0.3]}

    sp.sign_plot(scores, **heatmap_args)

# Load results as saved during computation

## SCORE METRICS

In [13]:
#load result files as specified, store in one large dictionary
dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "lendingclub", "hmeq", "australian", "german", "thomas", "pakdd"]
ratios_list = [0.1,0.2,0.3,0.4,0.5] #,0.2,0.3,0.4,0.5
cost_mat_list = [None] #only one cost setup was used, others are discarded due to unreliable results and computation issues
weights_list = ['wF', 'wT']
results_dict = {'wT': {}, 'wF': {}} #load results for parameter weights True/False separately

for w in weights_list:
  for d in dataset_list:
    results_dict[w][d] = {}
    for r in ratios_list:
      for c in cost_mat_list:
        if w == 'wT':
          filename = f"{d}__rounds-5_max_gens-10_AL_acc_rate-{r}_weights-True_cost_matrix-{c}_do_thres-tuned"
        else:
          filename = f"{d}__rounds-5_max_gens-10_AL_acc_rate-{r}_weights-False_cost_matrix-{c}_do_thres-tuned"
          
        filename = filename.replace('.', '-')
        filename = filename.replace('[', '')
        filename = filename.replace(']', '')
        filename = filename.replace(',', '-')
        filename = filename.replace(' ', '')

        infile = open(f'model_results/{d}/{filename}','rb')

        rname = f'r{r}'.replace('.', '')

        results_dict[w][d][rname] = pickle.load(infile)

        infile.close()
        #if fname not in configs_list:
        #  configs_list.append(fname)

#print(configs_list)

In [14]:
results_dict['wT']['bene2']['r05']['bmdr']['average']

,generation,n_accepts,n_rejects,bad_ratio_accepts,bad_ratio_rejects,n_score_accepts,n_AL_selects,bad_ratio_score_accepts,bad_ratio_AL_selects,roc_auc,brier,h-measure,average_precision,balanced_accuracy,pcc,precision,recall,f1,fpr,fnr,tn,fp,fn,tp
0,0.0,NaN,NaN,0.29160,0.30028,NaN,NaN,NaN,NaN,0.758830,0.177684,0.234831,0.540208,0.684491,0.578710,0.434962,0.840068,0.571969,0.471086,0.159932,532.4,474.2,69.0,362.4
1,1.0,711.6,288.4,0.26748,0.36842,105.8,105.8,0.07334,0.34204,0.766488,0.174078,0.245922,0.550996,0.696441,0.623819,0.456301,0.805753,0.582142,0.412871,0.194247,591.0,415.6,83.8,347.6
2,2.0,942.8,557.2,0.25054,0.37818,221.4,221.4,0.05932,0.34518,0.771724,0.172381,0.255411,0.557923,0.702277,0.645692,0.468263,0.791392,0.587698,0.386837,0.208608,617.2,389.4,90.0,341.4
3,3.0,1182.4,817.6,0.24414,0.37734,341.2,341.2,0.06006,0.35600,0.774873,0.171238,0.259986,0.557076,0.708305,0.664717,0.478932,0.782584,0.593705,0.365975,0.217416,638.2,368.4,93.8,337.6
4,4.0,1430.4,1069.6,0.23828,0.37978,465.2,465.2,0.06228,0.35520,0.776164,0.170777,0.260905,0.557443,0.708176,0.672700,0.483214,0.769611,0.593281,0.353259,0.230389,651.0,355.6,99.4,332.0
5,5.0,1688.8,1311.2,0.23446,0.38256,594.4,594.4,0.05972,0.35992,0.779108,0.169513,0.266901,0.564664,0.712842,0.679709,0.487433,0.775172,0.598350,0.349487,0.224828,654.8,351.8,97.0,334.4
6,6.0,1948.4,1551.6,0.23314,0.38218,724.2,724.2,0.06048,0.36456,0.778725,0.169652,0.266048,0.562293,0.710519,0.678371,0.486515,0.769135,0.595936,0.348098,0.230865,656.2,350.4,99.6,331.8
7,7.0,2202.8,1797.2,0.23172,0.38244,851.4,851.4,0.05842,0.36914,0.780182,0.169125,0.266702,0.565127,0.712605,0.682372,0.489043,0.770525,0.598226,0.345315,0.229475,659.0,347.6,99.0,332.4
8,8.0,2464.0,2036.0,0.23260,0.38028,982.0,982.0,0.05696,0.37772,0.780933,0.168638,0.267621,0.566897,0.710612,0.676819,0.485552,0.771905,0.596087,0.350681,0.228095,653.6,353.0,98.4,333.0
9,9.0,2733.2,2266.8,0.23302,0.37990,1116.6,1116.6,0.05640,0.38290,0.781396,0.168432,0.269041,0.567760,0.710745,0.674102,0.483976,0.776541,0.596265,0.355052,0.223459,649.2,357.4,96.4,335.0


## COST METRICS

In [15]:
#load result files as specified, store in one large dictionary
dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "lendingclub", "hmeq", "australian", "german", "thomas", "pakdd"]
cost_dict = {'wT': {}, 'wF': {}} #load results for parameter weights True/False separately
#configs_list = []

for w in weights_list:
  for d in dataset_list:
    cost_dict[w][d] = {}
    for r in ratios_list:
      for c in cost_mat_list:
        if w == 'wT':
          filename = f"{d}__rounds-5_max_gens-10_AL_acc_rate-{r}_weights-True_cost_matrix-{c}_do_thres-tuned"
        else:
          filename = f"{d}__rounds-5_max_gens-10_AL_acc_rate-{r}_weights-False_cost_matrix-{c}_do_thres-tuned"
          
        filename = filename.replace('.', '-')
        filename = filename.replace('[', '')
        filename = filename.replace(']', '')
        filename = filename.replace(',', '-')
        filename = filename.replace(' ', '')

        infile = open(f'model_cost/{d}/{filename}','rb')

        rname = f'r{r}'.replace('.', '')

        cost_dict[w][d][rname] = pickle.load(infile)

        infile.close()
        #if fname not in configs_list:
        #  configs_list.append(fname)

#print(configs_list)

In [16]:
cost_dict['wT']['bene2']['r05']['bmdr']['average']

,generation,gen_internal_cost,gen_internal_cpl,total_internal_cost,total_internal_cpl,model_internal_cost,model_internal_cpl,external_cost,external_cpl
0,0.0,340.200000,0.680400,340.200000,0.680400,0.000000,NaN,635.200000,0.441725
1,1.0,285.533333,0.571067,625.733333,0.625733,285.533333,0.571067,611.133333,0.424988
2,2.0,271.466667,0.542933,897.200000,0.598133,557.000000,0.557000,599.400000,0.416829
3,3.0,285.333333,0.570667,1182.533333,0.591267,842.333333,0.561556,587.266667,0.408391
4,4.0,276.200000,0.552400,1458.733333,0.583493,1118.533333,0.559267,587.533333,0.408577
5,5.0,275.000000,0.550000,1733.733333,0.577911,1393.533333,0.557413,578.133333,0.402040
6,6.0,285.266667,0.570533,2019.000000,0.576857,1678.800000,0.559600,582.800000,0.405285
7,7.0,282.333333,0.564667,2301.333333,0.575333,1961.133333,0.560324,578.600000,0.402364
8,8.0,298.333333,0.596667,2599.666667,0.577704,2259.466667,0.564867,582.600000,0.405146
9,9.0,292.866667,0.585733,2892.533333,0.578507,2552.333333,0.567185,582.333333,0.404961


## Build table template to fill with aggregated results

In [18]:
# ALT: EVERY PARAMETER COMBINATION AS OWN CLASSIFER
#model_list = ['oracle','score', 'random', 'unc', 'qbc', 'dw', 'cors', 'density', 'eer', 'lal', 'quire', 'bmdr', 'spal']
model_list = ['score', 'random', 'unc', 'qbc', 'dw', 'cors', 'density', 'eer', 'lal', 'quire', 'bmdr', 'spal']
performance_metrics = ['roc_auc', 'brier', 'h-measure', 'pcc', 'balanced_accuracy', 'f1']
cost_metrics = ['total_internal_cost','total_internal_cpl','external_cost','external_cpl']

# built list of column names
c_names = model_list

#column_names = ['score']
column_names = []

index_names = []

#################################
for m in model_list:
  for r in ["r01","r02","r03","r04","r05"]:
    for w in weights_list:
      column_names.append(f'{m}_{w}_{r}')
  #if m != 'score':
    #column_names.append(f'{m}_r05')
#################################

for d in dataset_list:
  for s in performance_metrics:
    for g in range(10):
      index_names.append(f'{d}_{s}_g{g+1}')
  for c in cost_metrics:
    for g in range(10):
      index_names.append(f'{d}_{c}_g{g+1}')

results_template = pd.DataFrame(columns=column_names, index=index_names)
results_template

,score_wF_r01,score_wT_r01,score_wF_r02,score_wT_r02,score_wF_r03,score_wT_r03,score_wF_r04,score_wT_r04,score_wF_r05,score_wT_r05,random_wF_r01,random_wT_r01,random_wF_r02,random_wT_r02,random_wF_r03,random_wT_r03,random_wF_r04,random_wT_r04,random_wF_r05,random_wT_r05,unc_wF_r01,unc_wT_r01,unc_wF_r02,unc_wT_r02,unc_wF_r03,unc_wT_r03,unc_wF_r04,unc_wT_r04,unc_wF_r05,unc_wT_r05,qbc_wF_r01,qbc_wT_r01,qbc_wF_r02,qbc_wT_r02,qbc_wF_r03,qbc_wT_r03,qbc_wF_r04,qbc_wT_r04,qbc_wF_r05,qbc_wT_r05,dw_wF_r01,dw_wT_r01,dw_wF_r02,dw_wT_r02,dw_wF_r03,dw_wT_r03,dw_wF_r04,dw_wT_r04,dw_wF_r05,dw_wT_r05,cors_wF_r01,cors_wT_r01,cors_wF_r02,cors_wT_r02,cors_wF_r03,cors_wT_r03,cors_wF_r04,cors_wT_r04,cors_wF_r05,cors_wT_r05,density_wF_r01,density_wT_r01,density_wF_r02,density_wT_r02,density_wF_r03,density_wT_r03,density_wF_r04,density_wT_r04,density_wF_r05,density_wT_r05,eer_wF_r01,eer_wT_r01,eer_wF_r02,eer_wT_r02,eer_wF_r03,eer_wT_r03,eer_wF_r04,eer_wT_r04,eer_wF_r05,eer_wT_r05,lal_wF_r01,lal_wT_r01,lal_wF_r02,lal_wT_r02,lal_wF_r03,lal_wT_r03,lal_wF_r04,lal_wT_r04,lal_wF_r05,lal_wT_r05,quire_wF_r01,quire_wT_r01,quire_wF_r02,quire_wT_r02,quire_wF_r03,quire_wT_r03,quire_wF_r04,quire_wT_r04,quire_wF_r05,quire_wT_r05,bmdr_wF_r01,bmdr_wT_r01,bmdr_wF_r02,bmdr_wT_r02,bmdr_wF_r03,bmdr_wT_r03,bmdr_wF_r04,bmdr_wT_r04,bmdr_wF_r05,bmdr_wT_r05,spal_wF_r01,spal_wT_r01,spal_wF_r02,spal_wT_r02,spal_wF_r03,spal_wT_r03,spal_wF_r04,spal_wT_r04,spal_wF_r05,spal_wT_r05
bene2_roc_auc_g1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bene2_roc_auc_g2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bene2_roc_auc_g3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bene2_roc_auc_g4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bene2_roc_auc_g5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

## Fill in results

In [19]:
# fill in values from results data
performance = copy.deepcopy(results_template)

for w in ['wT', 'wF']:
  for r in ["r01","r02","r03","r04","r05"]:
    for s in performance_metrics:
      for d in dataset_list:
        for g in range(10):
          for m in model_list:
            #print('RESULTS',w,r,s,d,g,m)
            performance.at[f'{d}_{s}_g{g+1}', f'{m}_{w}_{r}'] = results_dict[w][d][r][m]['average'][s][g+1]

    for c in cost_metrics:
      for d in dataset_list:
        for g in range(10):
          for m in model_list:
            #print('COST',w,r,c,d,g,m)

            performance.at[f'{d}_{c}_g{g+1}', f'{m}_{w}_{r}'] = cost_dict[w][d][r][m]['average'][c][g+1]



##################################

#for m in model_list:
  #if m == 'score':
    #performance.loc[:,'score'] = performance.filter(regex=f'{m}_').mean(axis=1)
  #else:
    #performance.loc[:,f'{m}_r05'] = performance.filter(regex=f'{m}_.*_r05').mean(axis=1) 
    #no need to differentiate between wT and wF when r = 0.5, since the dynamic (inversely proportional) sample weights will all be 1
    #(no difference between score and AL-selections in terms of sample weights)

'''#remove redundant columns (score model and any wT_r05 or wF_r05) before ranking models in order to not dilute ranks
for m in model_list:
  for w in ['wT', 'wF']:
    if m == 'score':
      for r in ["r01","r02","r03","r04","r05"]:
        performance = performance.drop(f'{m}_{w}_{r}', axis=1)
    else:    
      performance = performance.drop(f'{m}_{w}_r05', axis=1)

#build ranks
performance_ranks = performance.rank(ascending=False, axis="columns")
##invert ranks for metrics where less is better
performance_ranks.loc[performance_ranks.index.str.contains('brier|cost|cpl')] = performance.loc[performance.index.str.contains('brier|cost|cpl')].rank(ascending=True, axis="columns")

# make sure each score column has same values (re-adds columns, all with the same values and ranks)
for m in model_list:
  for w in ['wT', 'wF']:
    if m == 'score':
      for r in ["r01","r02","r03","r04","r05"]:
        performance[f'{m}_{w}_{r}'] = performance.loc[:,f'{m}']
        performance_ranks[f'{m}_{w}_{r}'] = performance_ranks.loc[:,f'{m}']
    else:
      performance[f'{m}_{w}_r05'] = performance.loc[:,f'{m}_r05']
      performance_ranks[f'{m}_{w}_r05'] = performance_ranks.loc[:,f'{m}_r05']'''


#performance = performance.reindex(sorted(performance.columns), axis=1)
#performance_ranks = performance_ranks.reindex(sorted(performance_ranks.columns), axis=1)

#performance_ranks.loc['mean_ranks']=(performance_ranks.loc[~performance_ranks.index.str.contains('internal')].mean())
#performance_ranks.loc['cost_ranks']=(performance_ranks.loc[performance_ranks.index.str.contains('external_cpl')].mean())

'#remove redundant columns (score model and any wT_r05 or wF_r05) before ranking models in order to not dilute ranks\nfor m in model_list:\n  for w in [\'wT\', \'wF\']:\n    if m == \'score\':\n      for r in ["r01","r02","r03","r04","r05"]:\n        performance = performance.drop(f\'{m}_{w}_{r}\', axis=1)\n    else:    \n      performance = performance.drop(f\'{m}_{w}_r05\', axis=1)\n\n#build ranks\nperformance_ranks = performance.rank(ascending=False, axis="columns")\n##invert ranks for metrics where less is better\nperformance_ranks.loc[performance_ranks.index.str.contains(\'brier|cost|cpl\')] = performance.loc[performance.index.str.contains(\'brier|cost|cpl\')].rank(ascending=True, axis="columns")\n\n# make sure each score column has same values (re-adds columns, all with the same values and ranks)\nfor m in model_list:\n  for w in [\'wT\', \'wF\']:\n    if m == \'score\':\n      for r in ["r01","r02","r03","r04","r05"]:\n        performance[f\'{m}_{w}_{r}\'] = performance.loc[:,f\

In [60]:
#performance.filter(regex=f'.*_r05')
performance.filter(regex=f'score').mean()

score_wF_r01    442.656063
score_wT_r01    442.656063
score_wF_r02    442.686977
score_wT_r02    442.686977
score_wF_r03    442.618641
score_wT_r03    442.618641
score_wF_r04    442.686010
score_wT_r04    442.686010
score_wF_r05    442.718969
score_wT_r05    442.718969
dtype: float64

In [101]:
#performance.loc[performance_ranks.index.str.contains('balance|external_cpl')].T

In [98]:
#testr = performance.filter(regex='r05').filter(regex='spal')
#np.where(testr['spal_wF_r05'] != testr['spal_wT_r05'])

(array([213, 215, 218, 219]),)

In [99]:
#testr.iloc[213]

spal_r05       0.169711
spal_wF_r05    0.169711
spal_wT_r05    0.169711
Name: bene2_brier_g4, dtype: object

In [ ]:
#performance_ranks

# Performance and Ranks (AVERAGE SCORES)

## Run Friedman and Post-Hoc tests for BEST PERFORMERS ONLY

In [61]:
"""
  Friedman Test with post-hoc test
"""

#metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc', 'balanced_accuracy', 'f1','total_internal_cost','total_internal_cpl','external_cost','external_cpl']
metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc','external_cpl', 'total_internal_cpl']
#metrics_list = ['balanced_accuracy', 'external_cpl']

dataset_list = ["australian", "bene1_nobins","bene2","german","gmsc", "hmeq","lendingclub", "pakdd", "uk", 'thomas'] #thomas
#dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "hmeq", "australian", "german"]
#dataset_list = ["lendingclub", "thomas", "pakdd"]


#############################################

#BUILD SUBSET OF RESULTS, based on metric and dataset choice
#then create a list of best-performing models, selecting each models' best-performing setup – either overall, or within a specified weight setup

#############################################

# create list of row names (indices), depending on choice of datasets and metrics
index_names = ''
for i in dataset_list:
  for j in metrics_list:
    index_names += f'{i}_{j}|'
index_names = index_names[:len(index_names)-1] #remove last | symbol
print(index_names)


#store subset of performance and rank dataframes (selected lines only)
performance_sub = performance.loc[performance.index.str.contains(index_names)]
#performance_sub = performance.loc[performance.index.str.contains('g1|g2|g3|g4|g5')]
#performance_sub = performance.loc[performance.index.str.contains('g6|g7|g8|g9|g10')]

###############
# AUGMENT DATA SUBSET for running tests

for m in model_list:
  if m == 'score':
    performance_sub['score'] = performance_sub.filter(regex=f'{m}_').mean(axis=1)
  if m != 'score':
    performance_sub.loc[:,f'{m}_r05'] = performance.filter(regex=f'{m}_.*_r05').mean(axis=1) 
    #no need to differentiate between wT and wF when r = 0.5, since the dynamic (inversely proportional) sample weights will all be 1
    #(no difference between score and AL-selections in terms of sample weights)


for m in model_list:
  for w in ['wT', 'wF']:
    if m == 'score':
      for r in ["r01","r02","r03","r04","r05"]:
        performance_sub = performance_sub.drop(f'{m}_{w}_{r}', axis=1)
    if m != 'score':    
      performance_sub = performance_sub.drop(f'{m}_{w}_r05', axis=1)

performance_sub = performance_sub.reindex(sorted(performance_sub.columns), axis=1)


###############

# build ranks after augmentation
performance_sub_ranks = performance_sub.rank(ascending=False, axis="columns")
##invert ranks for metrics where less is better
performance_sub_ranks.loc[performance_sub_ranks.index.str.contains('brier|cost|cpl')] = performance_sub.loc[performance_sub.index.str.contains('brier|cost|cpl')].rank(ascending=True, axis="columns")
#performance_sub_ranks = performance_ranks.loc[performance.index.str.contains(index_names)]

#before picking best-performing models for the overall comparison setup, drop any columns (models) that are redundant


performance_sub_ranks.loc['mean_ranks']=(performance_sub_ranks.loc[~performance_sub_ranks.index.str.contains('internal')].mean())
performance_sub_ranks.loc['cost_ranks']=(performance_sub_ranks.loc[performance_sub_ranks.index.str.contains('external_cpl')].mean())



###############
# get best-performing models, determined by average rank over datasets
# three setups: overall best, best by either weight setup
# option to pick best models either by averaging scores except internal cost, or by just picking external cost

#rank_choice = 'mean_ranks'
rank_choice = 'cost_ranks'

best_performers = {}
namelist = ['score']
#namelist = []
for m in model_list:
  if m != 'score':
    namelist.append(performance_sub_ranks.filter(regex=f'{m}').loc[rank_choice].idxmin())
best_performers['overall'] = namelist

for w in ['wT', 'wF']:
  namelist = ['score']
  #namelist = []
  for m in model_list:
    if m != 'score':
      namelist.append(performance_sub_ranks.filter(regex=f'{m}_{w}').loc[rank_choice].idxmin())
  best_performers[w] = namelist

print(best_performers)

performance_sub_ranks

australian_roc_auc|australian_brier|australian_h-measure|australian_pcc|australian_external_cpl|australian_total_internal_cpl|bene1_nobins_roc_auc|bene1_nobins_brier|bene1_nobins_h-measure|bene1_nobins_pcc|bene1_nobins_external_cpl|bene1_nobins_total_internal_cpl|bene2_roc_auc|bene2_brier|bene2_h-measure|bene2_pcc|bene2_external_cpl|bene2_total_internal_cpl|german_roc_auc|german_brier|german_h-measure|german_pcc|german_external_cpl|german_total_internal_cpl|gmsc_roc_auc|gmsc_brier|gmsc_h-measure|gmsc_pcc|gmsc_external_cpl|gmsc_total_internal_cpl|hmeq_roc_auc|hmeq_brier|hmeq_h-measure|hmeq_pcc|hmeq_external_cpl|hmeq_total_internal_cpl|lendingclub_roc_auc|lendingclub_brier|lendingclub_h-measure|lendingclub_pcc|lendingclub_external_cpl|lendingclub_total_internal_cpl|pakdd_roc_auc|pakdd_brier|pakdd_h-measure|pakdd_pcc|pakdd_external_cpl|pakdd_total_internal_cpl|uk_roc_auc|uk_brier|uk_h-measure|uk_pcc|uk_external_cpl|uk_total_internal_cpl|thomas_roc_auc|thomas_brier|thomas_h-measure|thomas_

,bmdr_r05,bmdr_wF_r01,bmdr_wF_r02,bmdr_wF_r03,bmdr_wF_r04,bmdr_wT_r01,bmdr_wT_r02,bmdr_wT_r03,bmdr_wT_r04,cors_r05,cors_wF_r01,cors_wF_r02,cors_wF_r03,cors_wF_r04,cors_wT_r01,cors_wT_r02,cors_wT_r03,cors_wT_r04,density_r05,density_wF_r01,density_wF_r02,density_wF_r03,density_wF_r04,density_wT_r01,density_wT_r02,density_wT_r03,density_wT_r04,dw_r05,dw_wF_r01,dw_wF_r02,dw_wF_r03,dw_wF_r04,dw_wT_r01,dw_wT_r02,dw_wT_r03,dw_wT_r04,eer_r05,eer_wF_r01,eer_wF_r02,eer_wF_r03,eer_wF_r04,eer_wT_r01,eer_wT_r02,eer_wT_r03,eer_wT_r04,lal_r05,lal_wF_r01,lal_wF_r02,lal_wF_r03,lal_wF_r04,lal_wT_r01,lal_wT_r02,lal_wT_r03,lal_wT_r04,qbc_r05,qbc_wF_r01,qbc_wF_r02,qbc_wF_r03,qbc_wF_r04,qbc_wT_r01,qbc_wT_r02,qbc_wT_r03,qbc_wT_r04,quire_r05,quire_wF_r01,quire_wF_r02,quire_wF_r03,quire_wF_r04,quire_wT_r01,quire_wT_r02,quire_wT_r03,quire_wT_r04,random_r05,random_wF_r01,random_wF_r02,random_wF_r03,random_wF_r04,random_wT_r01,random_wT_r02,random_wT_r03,random_wT_r04,score,spal_r05,spal_wF_r01,spal_wF_r02,spal_wF_r03,spal_wF_r04,spal_wT_r01,spal_wT_r02,spal_wT_r03,spal_wT_r04,unc_r05,unc_wF_r01,unc_wF_r02,unc_wF_r03,unc_wF_r04,unc_wT_r01,unc_wT_r02,unc_wT_r03,unc_wT_r04
bene2_roc_auc_g1,49.00,27.000,42.000,52.000,43.000,95.000,47.000,63.000,28.000,37.000,24.000,26.000,38.000,54.000,83.000,18.000,10.000,40.000,51.000,14.000,23.000,59.000,72.000,89.000,50.000,56.000,66.000,65.00,21.000,36.000,20.000,68.000,98.000,85.000,41.000,71.000,61.000,31.000,46.000,67.000,74.000,91.000,90.000,77.000,75.000,53.000,34.000,48.000,76.000,78.000,97.000,93.000,88.000,82.000,58.000,35.000,19.00,60.000,45.000,100.000,55.00,64.000,32.00,9.00,8.000,5.000,3.000,17.000,84.000,2.000,1.000,4.000,44.000,30.000,69.000,62.000,22.000,94.000,81.000,70.000,7.000,6.000,16.000,12.000,11.000,25.000,33.00,96.00,39.000,13.000,15.000,79.000,29.000,57.000,73.000,80.000,99.000,92.000,87.000,86.000
bene2_roc_auc_g2,39.00,24.000,17.000,44.000,55.000,85.000,37.000,53.000,42.000,59.000,32.000,40.000,46.000,56.000,73.000,60.000,21.000,49.000,36.000,23.000,38.000,43.000,61.000,89.000,64.000,57.000,50.000,63.00,28.000,34.000,22.000,62.000,86.000,41.000,14.000,67.000,78.000,30.000,68.000,74.000,72.000,98.000,93.000,88.000,81.000,80.000,51.000,52.000,71.000,79.000,97.000,83.000,87.000,91.000,33.000,31.000,27.00,48.000,35.000,99.000,25.00,54.000,26.00,6.00,10.000,4.000,8.000,11.000,70.000,1.000,2.000,3.000,66.000,47.000,45.000,69.000,29.000,96.000,65.000,76.000,12.000,19.000,9.000,20.000,18.000,13.000,16.00,94.00,15.000,5.000,7.000,82.000,58.000,75.000,77.000,84.000,100.000,95.000,92.000,90.000
bene2_roc_auc_g3,46.00,24.000,27.000,41.000,48.000,93.000,66.000,51.000,52.000,53.000,55.000,49.000,58.000,57.000,84.000,50.000,40.000,42.000,25.000,45.000,47.000,32.000,37.000,94.000,73.000,54.000,30.000,63.00,31.000,34.000,28.000,68.000,86.000,44.000,26.000,78.000,71.000,21.000,75.000,79.000,72.000,99.000,82.000,85.000,83.000,77.000,59.000,56.000,67.000,74.000,98.000,70.000,88.000,90.000,36.000,18.000,15.00,35.000,29.000,96.000,23.00,20.000,16.00,9.00,11.000,4.000,6.000,14.000,89.000,2.000,1.000,5.000,62.000,39.000,38.000,64.000,33.000,97.000,80.000,60.000,22.000,43.000,13.000,19.000,10.000,12.000,17.00,65.00,8.000,3.000,7.000,81.000,61.000,76.000,69.000,92.000,100.000,95.000,87.000,91.000
bene2_roc_auc_g4,50.00,32.000,26.000,55.000,49.000,97.000,81.000,47.000,53.000,29.000,51.000,41.000,52.000,43.000,78.000,21.000,28.000,13.000,30.000,58.000,63.000,54.000,45.000,95.000,73.000,44.000,27.000,61.00,33.000,37.000,36.000,64.000,83.000,38.000,39.000,70.000,79.000,23.000,80.000,85.000,84.000,100.000,75.000,91.000,88.000,74.000,59.000,62.000,67.000,66.000,93.000,71.000,68.000,82.000,24.000,20.000,25.00,57.000,35.000,94.000,19.00,18.000,6.00,4.00,15.000,11.000,9.000,16.000,96.000,17.000,3.000,7.000,60.000,34.000,56.000,48.000,40.000,98.000,65.000,22.000,42.000,46.000,5.000,31.000,12.000,8.000,10.00,86.00,14.000,2.000,1.000,76.000,69.000,77.000,72.000,87.000,99.000,92.000,90.000,89.000
bene2_roc_auc_g5,46.00,21.000,16.0

In [68]:

#############################################

#RUN TESTS

#############################################


friedman_results2 = {}
#nemenyi_results = {}
holm_results = {}
finner_results = {}
bon_dun_results = {}
#quade_scores = {}
friedman_statistics = pd.DataFrame(columns=['statistic', 'pvalue'], index=metrics_list)

for w in ['overall','wT', 'wF']:
  #nemenyi_results[w] = {}
  holm_results[w] = {}
  finner_results[w] = {}
  bon_dun_results[w] = {}
  for s in metrics_list:
    metric_values = []

    for d in dataset_list:
      row_values = []

      for m in best_performers[w]:
        row_values.append(performance_sub[f'{m}'].loc[performance_sub.index.str.contains(f'{d}_{s}')].mean())
        columns = best_performers[w]
        #print(columns)
      
      metric_values.append(row_values)

    
    #invert sign of measurements for metrics where more is better
    if s not in ['brier','total_internal_cost','total_internal_cpl','external_cost','external_cpl']:
      metric_values = np.array(metric_values) * -1
    
    print('\n\n\n------------------------\nMETRIC', s)
    #print(pd.DataFrame(metric_values, columns = columns))
    #print(scipy.stats.rankdata(pd.DataFrame(metric_values, columns = columns), axis=1))
    #print(scipy.stats.rankdata(pd.DataFrame(metric_values, columns = columns), axis=1).mean(axis=0))

    statistic, pvalue,statistic_uncorr, p_uncorr,ranking,ranking_avg, rank_cmp = friedman_test(*np.transpose(metric_values)) #this test applies the Iman-Davenport correction to
    #statistic, pvalue, ranking, ranking_avg, rank_cmp = friedmanchisquare2(*np.transpose(metric_values))

    print("average_ranks", ranking_avg)
    print("pivot", rank_cmp)

    temp = {'statistic':statistic, 'pvalue':pvalue}
    friedman_statistics.loc[f'{s}'] = temp

    # POST HOC TEST HOLM
    pivot_scores = {key: rank_cmp[i] for i, key in enumerate(columns)} 
    
    #get the name of the best-ranking model to use as control
    min_rank = pd.DataFrame(np.transpose(list(pivot_scores.values())), index = pivot_scores.keys()).iloc[:,0].argmin() 
    
    comparisons, z_values, p_values, adj_p_values = holm_test(pivot_scores, control=columns[min_rank])
    adj_p_values = np.asarray(adj_p_values)
    holm_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < 0.05}, index=comparisons)
    #print(holm_scores)
    holm_results[w][s] = copy.deepcopy(holm_scores)

    comparisons, z_values, p_values, adj_p_values = finner_test(pivot_scores, control=columns[min_rank])
    adj_p_values = np.asarray(adj_p_values)
    finner_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < 0.05}, index=comparisons)
    print(adj_p_values)
    print(finner_scores)
    finner_results[w][s] = copy.deepcopy(finner_scores)


    comparisons, z_values, p_values, adj_p_values = bonferroni_dunn_test(pivot_scores, control=columns[min_rank])
    adj_p_values = np.asarray(adj_p_values)
    bonferroni_dunn_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < 0.05}, index=comparisons)
    #print(bonferroni_dunn_scores)    
    bon_dun_results[w][s] = copy.deepcopy(bonferroni_dunn_scores)

    #nemenyi_results[w][s] = generate_scores(sp.posthoc_nemenyi_friedman, {}, metric_values, columns)
    #quade_scores[s] = generate_scores(sp.posthoc_quade, {}, metric_values, columns)

  print(f'Friedman results\n',friedman_statistics,'\n\n')
  friedman_results2[w] = copy.deepcopy(friedman_statistics)






------------------------
METRIC roc_auc
average_ranks [7.9, 5.5, 7.8, 3.7, 7.2, 8.0, 6.5, 6.1, 7.4, 5.6, 6.6, 5.7]
pivot [4.899372016273734, 3.4109552012032323, 4.8373546489791295, 2.294642589900356, 4.465250445211504, 4.961389383568338, 4.031128874149275, 3.7830594049708575, 4.589285179800712, 3.4729725684978363, 4.093146241443878, 3.534989935792441]
[0.08109503 0.08109503 0.08109503 0.08109503 0.08109503 0.1281913
 0.1281913  0.1829226  0.25593506 0.25914482 0.26428835]
                                     p    sig
qbc_wT_r04 vs cors_wF_r01     0.081095  False
qbc_wT_r04 vs score           0.081095  False
qbc_wT_r04 vs unc_wF_r01      0.081095  False
qbc_wT_r04 vs lal_wF_r02      0.081095  False
qbc_wT_r04 vs dw_wF_r03       0.081095  False
qbc_wT_r04 vs bmdr_wT_r04     0.128191  False
qbc_wT_r04 vs density_wT_r04  0.128191  False
qbc_wT_r04 vs eer_wT_r04      0.182923  False
qbc_wT_r04 vs spal_wF_r04     0.255935  False
qbc_wT_r04 vs quire_wT_r04    0.259145  False
qbc_wT_r04 vs 

In [ ]:
for m in model_list:
  print(performance_sub_ranks.loc['cost_ranks'].filter(regex=f'{m}').idxmin())

In [53]:
friedman_results2['overall'] #uncorrected pvalues (without thomas)

,statistic,pvalue
roc_auc,12.128205,0.354084
brier,34.179487,0.000338
h-measure,18.74359,0.065854
pcc,20.897436,0.034449
external_cpl,5.102564,0.926098
total_internal_cpl,72.282051,0.0


In [55]:
friedman_results2['overall'] #iman davenport pvalues (without thomas)

,statistic,pvalue
roc_auc,1.116883,0.358017
brier,4.218354,0.000051
h-measure,1.868371,0.054482
pcc,2.140512,0.025176
external_cpl,0.434735,0.936332
total_internal_cpl,21.642994,0.0


In [63]:
friedman_results2['overall'] #iman davenport pvalues (with thomas)

,statistic,pvalue
roc_auc,1.268071,0.254142
brier,6.588663,0.0
h-measure,2.146717,0.023508
pcc,3.023543,0.001624
external_cpl,0.547478,0.866073
total_internal_cpl,11.363924,0.0


In [48]:
friedman_results2['wT']

,statistic,pvalue
roc_auc,21.068376,0.03267
brier,31.34188,0.000972
h-measure,16.435897,0.125704
pcc,15.34188,0.167374
external_cpl,14.538462,0.204616
total_internal_cpl,47.42735,0.000002


In [49]:
friedman_results2['wF']

,statistic,pvalue
roc_auc,8.65812,0.653416
brier,22.555556,0.020406
h-measure,11.854701,0.374663
pcc,14.196581,0.222306
external_cpl,5.143024,0.924042
total_internal_cpl,66.709402,0.0


In [67]:
finner_results['overall']

{'brier':                                      p    sig
 qbc_wT_r04 vs score           0.000036   True
 qbc_wT_r04 vs cors_wF_r01     0.000044   True
 qbc_wT_r04 vs unc_wF_r01      0.004613   True
 qbc_wT_r04 vs lal_wF_r02      0.006517   True
 qbc_wT_r04 vs spal_wF_r04     0.013933   True
 qbc_wT_r04 vs quire_wT_r04    0.063193  False
 qbc_wT_r04 vs bmdr_wT_r04     0.096917  False
 qbc_wT_r04 vs density_wT_r04  0.126977  False
 qbc_wT_r04 vs random_wT_r04   0.206535  False
 qbc_wT_r04 vs dw_wF_r03       0.209896  False
 qbc_wT_r04 vs eer_wT_r04      0.709815  False,
 'external_cpl':                                      p    sig
 qbc_wT_r04 vs cors_wF_r01     0.612044  False
 qbc_wT_r04 vs score           0.612044  False
 qbc_wT_r04 vs spal_wF_r04     0.612044  False
 qbc_wT_r04 vs dw_wF_r03       0.612044  False
 qbc_wT_r04 vs bmdr_wT_r04     0.612044  False
 qbc_wT_r04 vs unc_wF_r01      0.612044  False
 qbc_wT_r04 vs lal_wF_r02      0.612044  False
 qbc_wT_r04 vs density_wT_r04  0.6

In [65]:
finner_results['wT']

{'brier':                                      p    sig
 eer_wT_r04 vs score           0.000049   True
 eer_wT_r04 vs lal_wT_r03      0.023604   True
 eer_wT_r04 vs spal_wT_r04     0.023604   True
 eer_wT_r04 vs quire_wT_r04    0.042255   True
 eer_wT_r04 vs dw_wT_r03       0.047234   True
 eer_wT_r04 vs cors_wT_r04     0.054241  False
 eer_wT_r04 vs bmdr_wT_r04     0.054416  False
 eer_wT_r04 vs random_wT_r04   0.126977  False
 eer_wT_r04 vs density_wT_r04  0.164374  False
 eer_wT_r04 vs unc_wT_r03      0.167763  False
 eer_wT_r04 vs qbc_wT_r04      0.950549  False,
 'external_cpl':                                      p    sig
 qbc_wT_r04 vs unc_wT_r03      0.056342  False
 qbc_wT_r04 vs dw_wT_r03       0.097302  False
 qbc_wT_r04 vs lal_wT_r03      0.097302  False
 qbc_wT_r04 vs spal_wT_r04     0.237830  False
 qbc_wT_r04 vs cors_wT_r04     0.237830  False
 qbc_wT_r04 vs score           0.263661  False
 qbc_wT_r04 vs density_wT_r04  0.285776  False
 qbc_wT_r04 vs eer_wT_r04      0.2

In [66]:
finner_results['wF']

{'brier':                                     p    sig
 dw_wF_r03 vs score           0.001703   True
 dw_wF_r03 vs cors_wF_r01     0.006911   True
 dw_wF_r03 vs unc_wF_r01      0.162439  False
 dw_wF_r03 vs quire_wF_r02    0.163389  False
 dw_wF_r03 vs lal_wF_r02      0.247104  False
 dw_wF_r03 vs random_wF_r03   0.263661  False
 dw_wF_r03 vs density_wF_r03  0.494577  False
 dw_wF_r03 vs spal_wF_r04     0.494577  False
 dw_wF_r03 vs bmdr_wF_r04     0.525637  False
 dw_wF_r03 vs eer_wF_r02      0.654842  False
 dw_wF_r03 vs qbc_wF_r04      0.804081  False,
 'external_cpl':                                         p    sig
 random_wF_r03 vs eer_wF_r02      0.711534  False
 random_wF_r03 vs cors_wF_r01     0.735608  False
 random_wF_r03 vs density_wF_r03  0.735608  False
 random_wF_r03 vs bmdr_wF_r04     0.735608  False
 random_wF_r03 vs spal_wF_r04     0.735608  False
 random_wF_r03 vs score           0.735608  False
 random_wF_r03 vs lal_wF_r02      0.735608  False
 random_wF_r03 vs dw_w

In [ ]:
holm_results['overall']

In [ ]:
nemenyi_results

In [ ]:
'''for k1 in nemenyi_results.keys():
  for k2 in nemenyi_results[k1].keys():
    pairwise_plotter(nemenyi_results[k1][k2])'''

pairwise_plotter(nemenyi_results['wF']['roc_auc'])

## Run Friedman Tests for 2x60 models (average SCORES)

In [58]:
"""
  Runs test comparing ALL models for each weight setup
"""


#model_list = ['score', 'random', 'unc', 'qbc', 'dw', 'cors', 'density', 'eer', 'lal', 'quire', 'bmdr', 'spal']
#metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc', 'balanced_accuracy', 'f1','total_internal_cost','total_internal_cpl','external_cost','external_cpl']
metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc','balanced_accuracy', 'f1', 'total_internal_cpl','external_cpl']  #'brier', 'h-measure', 'pcc',
dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "lendingclub", "hmeq", "pakdd", "australian", "german"] #,"thomas", 
#dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "hmeq", "australian", "german"]
#dataset_list = ["lendingclub", "thomas", "pakdd"]


friedman_results = {'wT':{}, 'wF':{}}
nemenyi_scores = {}
quade_scores = {}

for w in ['wT', 'wF']:
  friedman_results[w] = {}
  friedman_statistics = pd.DataFrame(columns=['statistic', 'pvalue'], index=metrics_list)

  for s in metrics_list:
    metric_values = []

    for d in dataset_list:
      row_values = []
      columns = []

      for m in model_list:
        row_values.extend(performance.filter(regex=f'{m}_{w}').loc[performance.index.str.contains(f'{d}_{s}')].mean().tolist())
        columns.extend(list(performance.filter(regex=f'{m}_{w}').columns))
      
      metric_values.append(row_values)

    #invert sign of measurements for metrics where more is better – needed because friedmanchisquare() uses scipy.stats.rankdata(), which ranks in ascending fashion (low value -> low rank)
    if s not in ['brier','total_internal_cost','total_internal_cpl','external_cost','external_cpl']:
      metric_values = np.array(metric_values) * -1

    print('\n\n\n------------------------\nMETRIC', s)

    #statistic, pvalue,statistic_uncorr, p_uncorr,ranking,ranking_avg, rank_cmp = friedman_test(*np.transpose(metric_values))
    statistic, pvalue, ranking, ranking_avg, rank_cmp = friedmanchisquare2(*np.transpose(metric_values))

    print('\n-------------------------------\nscipy Friedman MODDED\n-------------------------------\n')
    for pr in [statistic, pvalue, ranking_avg, rank_cmp]: #, ranking,ranking_avg, rank_cmp]:
      print(pr)

    temp = {'statistic':statistic, 'pvalue':pvalue}
    friedman_statistics.loc[f'{s}'] = temp



    # POST HOC TEST HOLM
    ranks = {key: rank_cmp[i] for i, key in enumerate(columns)} 
    min_rank = pd.DataFrame(np.transpose(list(ranks.values())), index = ranks.keys()).iloc[:,0].argmin()
    
    comparisons, z_values, p_values, adj_p_values = holm_test(ranks, control=columns[min_rank])
    adj_p_values = np.asarray(adj_p_values)
    holm_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < 0.05}, index=comparisons)
    print(holm_scores)    


    comparisons, z_values, p_values, adj_p_values = finner_test(pivot_scores, control=columns[min_rank])
    adj_p_values = np.asarray(adj_p_values)
    finner_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < 0.05}, index=comparisons)
    print(finner_scores)
    #finner_results[w][s] = copy.deepcopy(finner_scores)


    #nemenyi_scores[s] = generate_scores(sp.posthoc_nemenyi_friedman, {}, metric_values, columns)
    #quade_scores[s] = generate_scores(sp.posthoc_quade, {}, metric_values, columns)


  print(np.shape(metric_values))
  print(f'Friedman results for {w}\n',friedman_statistics,'\n\n')
  friedman_results[w][r] = friedman_statistics

'''for key in nemenyi_scores.keys():
  pairwise_plotter(nemenyi_scores[key])'''





------------------------
METRIC roc_auc

-------------------------------
scipy Friedman MODDED
-------------------------------

275.40295307818246
1.1040457182485505e-29
[24.77777777777778, 24.77777777777778, 24.77777777777778, 24.77777777777778, 24.77777777777778, 51.111111111111114, 34.666666666666664, 25.88888888888889, 17.666666666666668, 21.444444444444443, 54.22222222222222, 38.0, 31.88888888888889, 28.77777777777778, 22.88888888888889, 55.111111111111114, 32.111111111111114, 21.555555555555557, 12.88888888888889, 16.0, 55.44444444444444, 42.0, 28.555555555555557, 25.555555555555557, 22.666666666666668, 47.888888888888886, 38.111111111111114, 24.555555555555557, 16.88888888888889, 17.444444444444443, 53.77777777777778, 35.22222222222222, 24.555555555555557, 16.77777777777778, 15.333333333333334, 54.22222222222222, 39.888888888888886, 29.666666666666668, 22.22222222222222, 21.77777777777778, 54.0, 38.111111111111114, 28.666666666666668, 23.88888888888889, 18.444444444444443, 47

ValueError: ignored

In [ ]:
columns = []
columns.append(performance.filter(regex='bmdr_wT').columns)
type(columns)

list

In [ ]:
 performance.filter(regex=f'{w}_{r}').columns

Index(['bmdr_wF_r05', 'cors_wF_r05', 'density_wF_r05', 'dw_wF_r05',
       'eer_wF_r05', 'lal_wF_r05', 'qbc_wF_r05', 'quire_wF_r05',
       'random_wF_r05', 'score_wF_r05', 'spal_wF_r05', 'unc_wF_r05'],
      dtype='object')

In [ ]:
pd.DataFrame(np.transpose(list(ranks.values())), index = ranks.keys())

,0
score_wF_r05,2.668498
random_wF_r05,3.780372
unc_wF_r05,3.520934
qbc_wF_r05,3.261497
dw_wF_r05,3.854496
cors_wF_r05,3.632122
density_wF_r05,3.039122
eer_wF_r05,3.150310
lal_wF_r05,4.521621
quire_wF_r05,2.446123


In [ ]:
ranks

{'bmdr_wF_r05': 2.816747403312185,
 'cors_wF_r05': 3.632121651639396,
 'density_wF_r05': 3.0391221983105146,
 'dw_wF_r05': 3.8544964466377265,
 'eer_wF_r05': 3.15030959580968,
 'lal_wF_r05': 4.521620831632717,
 'qbc_wF_r05': 3.261496993308845,
 'quire_wF_r05': 2.446122744981634,
 'random_wF_r05': 3.780371514971616,
 'score_wF_r05': 2.6684975399799646,
 'spal_wF_r05': 3.780371514971616,
 'unc_wF_r05': 3.5209342541402306}

In [ ]:
friedman_results

{'wF': {'r01':                     statistic    pvalue
  roc_auc             11.725275  0.384644
  brier               21.263736  0.030739
  h-measure            5.241758  0.918895
  pcc                  7.263736  0.777341
  balanced_accuracy   11.021978  0.441424
  f1                   9.461538   0.57937
  total_internal_cpl  40.538462  0.000029
  external_cpl        11.021978  0.441424,
  'r02':                     statistic    pvalue
  roc_auc              7.967033  0.716254
  brier               21.857143  0.025504
  h-measure            6.956044  0.802632
  pcc                  8.868132  0.634063
  balanced_accuracy    9.725275  0.555255
  f1                   5.285714  0.916544
  total_internal_cpl  43.175824   0.00001
  external_cpl         9.725275  0.555255,
  'r03':                     statistic    pvalue
  roc_auc              6.802198  0.814868
  brier               19.769231  0.048608
  h-measure            4.296703   0.96042
  pcc                  9.527473  0.573323
  bal

## run separate tests for each combination of weights and ratio

In [ ]:
#Friedman Test with post-hoc test
#model_list = ['score', 'random', 'unc', 'qbc', 'dw', 'cors', 'density', 'eer', 'lal', 'quire', 'bmdr', 'spal']
#metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc', 'balanced_accuracy', 'f1','total_internal_cost','total_internal_cpl','external_cost','external_cpl']
metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc','balanced_accuracy', 'f1', 'total_internal_cpl','external_cpl']  #'brier', 'h-measure', 'pcc',
dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "lendingclub", "hmeq", "pakdd"] #"australian", "german", "thomas", 
#dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "hmeq", "australian", "german"]
#dataset_list = ["lendingclub", "thomas", "pakdd"]


friedman_results = {'wT':{}, 'wF':{}}
nemenyi_scores = {}
quade_scores = {}

average_ratios = False #this feature does not work correctly, if this is enabled it is unnecessary to loop over ratios. 

for w in ['wT', 'wF']:
  friedman_results[w] = {}
  
  for r in ["r01","r02","r03","r04","r05"]:
    friedman_statistics = pd.DataFrame(columns=['statistic', 'pvalue'], index=metrics_list)

    for s in metrics_list:
      metric_values = []

      for d in dataset_list:
        row_values = []
        if average_ratios:
          columns = model_list
        else:
          columns = []

        for m in model_list:
          if average_ratios:
            row_values.append(performance.filter(regex=f'{m}_{w}').loc[performance.index.str.contains(f'{d}_{s}')].mean().mean())
          else:
            row_values.extend(performance.filter(regex=f'{m}_{w}_{r}').loc[performance.index.str.contains(f'{d}_{s}')].mean().tolist())
            columns.append(f'{m}_{w}_{r}')
        
        metric_values.append(row_values)

      #invert sign of measurements for metrics where more is better – needed because friedmanchisquare() uses scipy.stats.rankdata(), which ranks in ascending fashion (low value -> low rank)
      if s not in ['brier','total_internal_cost','total_internal_cpl','external_cost','external_cpl']:
        metric_values = np.array(metric_values) * -1

      print('\n\n\n------------------------\nMETRIC', s)

      #statistic, pvalue,statistic_uncorr, p_uncorr,ranking,ranking_avg, rank_cmp = friedman_test(*np.transpose(metric_values))
      statistic, pvalue, ranking, ranking_avg, rank_cmp = friedmanchisquare2(*np.transpose(metric_values))

      print('\n-------------------------------\nscipy Friedman MODDED\n-------------------------------\n')
      for pr in [statistic, pvalue, ranking_avg, rank_cmp]: #, ranking,ranking_avg, rank_cmp]:
        print(pr)

      print('\n-------------------------------\nCustom STAC\n-------------------------------\n')
      print(friedman_test(*np.transpose(metric_values)))

      temp = {'statistic':statistic, 'pvalue':pvalue}
      friedman_statistics.loc[f'{s}'] = temp



      # POST HOC TEST HOLM
      ranks = {key: rank_cmp[i] for i, key in enumerate(columns)} 
      print('ranking_avg', ranking_avg)
      print(pd.DataFrame(np.transpose(list(ranks.values())), index = ranks.keys()).iloc[:,0])
      min_rank = pd.DataFrame(np.transpose(list(ranks.values())), index = ranks.keys()).iloc[:,0].argmin()
      
      comparisons, z_values, p_values, adj_p_values = holm_test(ranks, control=columns[min_rank])
      adj_p_values = np.asarray(adj_p_values)
      holm_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < 0.05}, index=comparisons)
      print(holm_scores)    

      #nemenyi_scores[s] = generate_scores(sp.posthoc_nemenyi_friedman, {}, metric_values, columns)
      #quade_scores[s] = generate_scores(sp.posthoc_quade, {}, metric_values, columns)


    print(np.shape(metric_values))
    print(f'Friedman results for {w}\n',friedman_statistics,'\n\n')
    friedman_results[w][r] = friedman_statistics

'''for key in nemenyi_scores.keys():
  pairwise_plotter(nemenyi_scores[key])'''





------------------------
METRIC roc_auc

-------------------------------
scipy Friedman MODDED
-------------------------------

25.571428571428612
0.0075125990565410835
[1.4285714285714286, 5.857142857142857, 8.0, 7.714285714285714, 8.0, 6.428571428571429, 6.142857142857143, 9.285714285714286, 7.428571428571429, 4.428571428571429, 7.714285714285714, 5.571428571428571]
[0.7412493166611012, 3.0391221983105146, 4.1509961733021665, 4.002746309969947, 4.1509961733021665, 3.3356219249749555, 3.1873720616427352, 4.818120558297158, 3.8544964466377265, 2.297872881649414, 4.002746309969947, 2.8908723349782943]

-------------------------------
Custom STAC
-------------------------------

(2.9833333333333334, 0.002844721456970456, 25.571428571428573, 0.0075125990565411945, [[1.0, 6.0, 12.0, 10.0, 3.0, 2.0, 7.0, 11.0, 9.0, 5.0, 8.0, 4.0], [3.0, 8.0, 7.0, 9.0, 11.0, 1.0, 5.0, 2.0, 4.0, 10.0, 6.0, 12.0], [1.0, 7.0, 10.0, 6.0, 11.0, 8.0, 2.0, 9.0, 12.0, 3.0, 5.0, 4.0], [1.0, 2.0, 10.0, 4.0, 9.0, 7.

'for key in nemenyi_scores.keys():\n  pairwise_plotter(nemenyi_scores[key])'